In [1]:
import networkx as nx
import random
import copy
import time
import matplotlib.pyplot as plt
import sys
from collections import defaultdict
%matplotlib inline

def progressBar(value, endvalue, bar_length=20):

        percent = float(value) / endvalue
        arrow = '-' * int(round(percent * bar_length)-1) + '>'
        spaces = ' ' * (bar_length - len(arrow))

        sys.stdout.write("\rCalculating: [{0}] {1}%".format(arrow + spaces, int(round(percent * 100))))
        sys.stdout.flush()

        
def kawai_draw(G, infected=[]) :
    colors = []
    for node in G :
        if node in infected :
            colors.append('red')
        else :
            colors.append('green')
    nx.draw(G, layout=nx.kamada_kawai_layout(G), node_color=colors, node_size = 30, alpha = 0.7)
    plt.show()
    
num_nodes = 2000
    
G = nx.scale_free_graph(num_nodes).to_undirected()

#G = nx.watts_strogatz_graph(2000, 10, 1)

In [2]:
#kawai_draw(G)

In [3]:
import networkx as nx
import copy
import time
start = time.time()

spl = nx.all_pairs_shortest_path_length(G)

def change_ds(spl_gen, num_nodes, max_radius):
    ''' Transforms 'spl_gen' into the accepted datastructure
        by the 'infected_average_by_separation_degree' method
    '''
    result = []
    for n in range(num_nodes):
        progressBar(n, num_nodes, 50)
        di_tmp = {}
        for entry in spl_gen:  
            for radius in range(1, max_radius + 1):
                li_tmp = []
                for node, dist in entry[1].items():
                    if dist == radius:
                        li_tmp.append(node)
                if li_tmp != []:
                    di_tmp[radius - 1] = li_tmp
            result.append(copy.copy(di_tmp))
            
    return result

separation_degrees = change_ds(spl, num_nodes, 6)
print('\n', time.time() - start)

Calculating: [------------------------------------------------->] 100%
 17.120984315872192


In [4]:
def infected_percentage(nodes, infected) :
    count = 0
    for node in nodes :
        if node in infected :
            count += 1
    
    return count / len(nodes)

def infected_average_by_separation_degree(G, separation_degrees, infected) :
    node_count = {}
    percentage_sum = {}
    
    for node in range(0, len(separation_degrees)) :
        if node in infected :
            for degree in separation_degrees[node] :
                if degree not in percentage_sum.keys() :
                    percentage_sum[degree] = 0
                    node_count[degree] = 0
                percentage_sum[degree] += infected_percentage(separation_degrees[node][degree], infected)
                node_count[degree] += 1
        
    average_by_separation_degree = {}
    
    for separation_degree in percentage_sum :
        average_by_separation_degree[separation_degree] = percentage_sum[separation_degree] / node_count[separation_degree]
            
    return average_by_separation_degree


In [7]:
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics.SIModel as si
import ndlib.models.opinions.VoterModel as vt

# Model selection
model = si.SIModel(G)
#model = vt.VoterModel(G)

# Model Configuration
cfg = mc.Configuration()
cfg.add_model_parameter('beta', 0.01) # prob of infecteded
cfg.add_model_parameter("percentage_infected", 0.5) # initially infected  %
model.set_initial_status(cfg)

# Simulation execution
iterations = model.iteration_bunch(60)

In [8]:
show_degrees = 10
infected = []
infected_count = []
avg_traits = []
for it in range(0, len(iterations)) :
    print('Iteration', it+1)
    for node, i in iterations[it]['status'].items() :
        if i == 1:
            infected.append(node)
            
    print('Infected count', len(infected))
    #kawai_draw(G)
    percentages = infected_average_by_separation_degree(G, separation_degrees, infected)
    
    print('Separations:')
    to_show = []
    for d in range(0, show_degrees) :
        if d in percentages :
            to_show.append(percentages[d])
        else :
            break
    
    avg_traits.append(percentages) # needed for propensity
    infected_count.append(len(infected)) # needed for the random graph generation
    
    print(to_show)

Iteration 1
Infected count 1000
Separations:
[0.5618313501404059, 0.5082279330593185, 0.49708229208253774, 0.48698795870637607, 0.5617106418858099, 0.6357843263437845]
Iteration 2
Infected count 1001
Separations:
[0.5626020813923469, 0.507720212846472, 0.49732177575924674, 0.4889981376899035, 0.5622143351924609, 0.6358533830085659]
Iteration 3
Infected count 1001
Separations:
[0.5626020813923469, 0.507720212846472, 0.49732177575924674, 0.4889981376899035, 0.5622143351924609, 0.6358533830085659]
Iteration 4
Infected count 1002
Separations:
[0.5646810205821636, 0.5083220410600423, 0.49770866413263953, 0.4891062661619541, 0.5621961479809875, 0.6352104574443509]
Iteration 5
Infected count 1003
Separations:
[0.5651757020779408, 0.5086626875126457, 0.49847462852479163, 0.48934655823805234, 0.5622967342526044, 0.635582506184225]
Iteration 6
Infected count 1003
Separations:
[0.5651757020779408, 0.5086626875126457, 0.49847462852479163, 0.48934655823805234, 0.5622967342526044, 0.635582506184225]

Separations:
[0.571475619954544, 0.5136309974610783, 0.5036455816709356, 0.5005920390101051, 0.5657265540205703, 0.638101295188347]
Iteration 52
Infected count 1028
Separations:
[0.571475619954544, 0.5136309974610783, 0.5036455816709356, 0.5005920390101051, 0.5657265540205703, 0.638101295188347]
Iteration 53
Infected count 1029
Separations:
[0.5715820215845723, 0.5146075382564794, 0.5039476756756975, 0.5007037718452901, 0.5657351113162623, 0.637963744097522]
Iteration 54
Infected count 1029
Separations:
[0.5715820215845723, 0.5146075382564794, 0.5039476756756975, 0.5007037718452901, 0.5657351113162623, 0.637963744097522]
Iteration 55
Infected count 1029
Separations:
[0.5715820215845723, 0.5146075382564794, 0.5039476756756975, 0.5007037718452901, 0.5657351113162623, 0.637963744097522]
Iteration 56
Infected count 1029
Separations:
[0.5715820215845723, 0.5146075382564794, 0.5039476756756975, 0.5007037718452901, 0.5657351113162623, 0.637963744097522]
Iteration 57
Infected count 1030
Separa

In [9]:
''' Start Random distribution of infecteds section '''
import random
from collections import defaultdict
from copy import deepcopy # only used for testing

def simulate_random_infecteds(G_rand, num_nodes, radius_split_rand, infected_count):
    
    '''
    Randomly picks the same number of infecteds as in the 
    non-random model (for a each iteration of the spreading)
    Returns a list of @infected_average_by_radius lists
    '''
    print(infected_count)
    random_infected = []
    random_not_infected = list(range(0, num_nodes))
    prev_num_infects = 0
    new_num_infects = 0
    avg_trait_sharing_by_radius = []
    for num_infects in infected_count:
        print("num_infects: ", num_infects)
        new_num_infects = num_infects - prev_num_infects
        print("new infecteds: ", new_num_infects)
        # prev_random_infected = deepcopy(random_infected) # for testing purposes

        for _ in range(new_num_infects):
            new_inf = random.choice(random_not_infected)
            random_infected.append(new_inf)
            random_not_infected.remove(new_inf)
            
        prev_num_infects = num_infects
        percentage = infected_average_by_separation_degree(G_rand, separation_degrees, random_infected)
        avg_trait_sharing_by_radius.append(percentage)
        print(percentage)    
        
        # print("¨¨¨¨¨¨¨¨¨Infected list: ", len(random_infected), " nodes\n", random_infected)
        # print("¨¨¨¨¨¨¨¨¨NOT Infected list:", len(random_not_infected), " nodes\n", random_not_infected) 
        # print_list_differences(prev_random_infected, random_infected)
    return (avg_trait_sharing_by_radius, random_infected)

avg_random_trait_sharing_by_radius, final_random_infected = simulate_random_infecteds(G, num_nodes, separation_degrees, infected_count)

[1000, 1001, 1001, 1002, 1003, 1003, 1003, 1003, 1003, 1004, 1005, 1005, 1005, 1005, 1005, 1006, 1006, 1006, 1006, 1007, 1008, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1015, 1015, 1016, 1017, 1018, 1019, 1019, 1020, 1020, 1021, 1022, 1023, 1024, 1025, 1025, 1026, 1026, 1027, 1027, 1027, 1028, 1028, 1028, 1029, 1029, 1029, 1029, 1030, 1030, 1031, 1032]
num_infects:  1000
new infecteds:  1000
{0: 0.6944969566266794, 1: 0.5390697271440968, 2: 0.503905462814408, 3: 0.5016195778698788, 4: 0.3774854616720755, 5: 0.21076343181272247}
num_infects:  1001
new infecteds:  1
{0: 0.6948034079238998, 1: 0.5394799088822707, 2: 0.5043333330883365, 3: 0.5016969781781924, 4: 0.3774188927190317, 5: 0.2110813278539008}
num_infects:  1001
new infecteds:  0
{0: 0.6948034079238998, 1: 0.5394799088822707, 2: 0.5043333330883365, 3: 0.5016969781781924, 4: 0.3774188927190317, 5: 0.2110813278539008}
num_infects:  1002
new infecteds:  1
{0: 0.6951100362491006, 1: 0.5398320369305643, 2: 0.5048799603981663, 3

{0: 0.6994800032071895, 1: 0.5540676577634274, 2: 0.5171420363241472, 3: 0.5130554652368448, 4: 0.38148790325961557, 5: 0.21175725513730298}
num_infects:  1027
new infecteds:  1
{0: 0.7010721239815522, 1: 0.5544761718056402, 2: 0.5175692179264776, 3: 0.5131295729331334, 4: 0.38141912022391244, 5: 0.21179500710656526}
num_infects:  1027
new infecteds:  0
{0: 0.7010721239815522, 1: 0.5544761718056402, 2: 0.5175692179264776, 3: 0.5131295729331334, 4: 0.38141912022391244, 5: 0.21179500710656526}
num_infects:  1027
new infecteds:  0
{0: 0.7010721239815522, 1: 0.5544761718056402, 2: 0.5175692179264776, 3: 0.5131295729331334, 4: 0.38141912022391244, 5: 0.21179500710656526}
num_infects:  1028
new infecteds:  1
{0: 0.7018492911761226, 1: 0.5549103921646376, 2: 0.5178583913154516, 3: 0.5149580543963078, 4: 0.38224060267441834, 5: 0.21198853264603293}
num_infects:  1028
new infecteds:  0
{0: 0.7018492911761226, 1: 0.5549103921646376, 2: 0.5178583913154516, 3: 0.5149580543963078, 4: 0.382240602674

In [10]:
def propensity_by_radius(iteration, radius, avg_traits, rand_avg_traits):
    return (avg_traits[iteration-1][radius-1] / rand_avg_traits[iteration-1][radius-1]) - 1

def all_propensities(iteration, max_radius, avg_traits, rand_avg_traits):
    all_propensities = []
    print(iteration)
    for r in range(1, max_radius + 1):
        print(r)
        
        all_propensities.append(propensity_by_radius(iteration, r, avg_traits, rand_avg_traits))
    return all_propensities

prop_one = propensity_by_radius(10, 1, avg_traits, avg_random_trait_sharing_by_radius)
prop_two = propensity_by_radius(10, 2, avg_traits, avg_random_trait_sharing_by_radius)
print(prop_one)

all_props = all_propensities(10, 6, avg_traits, avg_random_trait_sharing_by_radius)
print(all_props)

def normalized_propensities(max_radius, all_propensities):
    normalized_propensities = []
    for r in range(max_radius):
        normalized_propensities.append(all_propensities[r] / all_propensities[0])
    
    return normalized_propensities
        
print(normalized_propensities(6, all_props))

-0.18603617471465872
10
1
2
3
4
5
6
[-0.18603617471465872, -0.05975272957273092, -0.014723091181829684, -0.027317356054614605, 0.4884314820964828, 2.0061748436254807]
[1.0, 0.3211887669931901, 0.07914101224889128, 0.14683894729889935, -2.625465089494752, -10.783788941599884]


In [11]:
#Do not delete, create list for each degree and add to report
avg = 0
for node in separation_degrees :
    if len(node) > 5 :
        avg += len(node[5])

print(avg / len(separation_degrees))

21.2945


In [12]:
print(2 * G.number_of_edges() / G.number_of_nodes())

4.387


In [13]:
'''import pickle

with open('2000_nodes_scale_free.graph', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(G, f)'''

"import pickle\n\nwith open('2000_nodes_scale_free.graph', 'wb') as f:  # Python 3: open(..., 'wb')\n    pickle.dump(G, f)"

In [ ]:
'''import pickle

with open('2000_nodes_scale_free.graph', 'rb') as f:  # Python 3: open(..., 'rb')
    Gtest = pickle.load(f)

print(2 * Gtest.number_of_edges() / Gtest.number_of_nodes())'''